# Evaluation metrics

## K-means shortcomings

K-means is a simple, fairly efficient, and commonly used clustering/partitioning algorithm.  It does however have some shortcomings that today's lecture will address. Mainly:

* How to choose K?
* Curse of dimensionality (using other similarity metrics)


The main difference being step `3.`, instead of simply moving the centroid to the mean of the assigned points we have to compute the cost of assigning a different data point (of the cluster) to be the new medoid.  We pick the new assignment that minimizes the cost.

## Choosing K

One of the unsolved problems in unsupervised learning is determining the ideal value for certain hyperparameters.  For clustering (and topic modelling), this is often a parameter `K` that specifies apriori how many groups you believe to exist in the data.

### Within cluster Dispersion

Often the most basic metric of how well your clustering alorgithm is performing is the within-cluster dispersion.

$$D_{r} = \sum_{i, i' \in C_{r}} d_{ii'}$$

$$W_{k} = \sum_{r = 1}^{k} \frac{1}{2n_{r}}D_{r}$$

The difficulty with this metric is that as you increase $k$, $W_{k}$ monotonically decreases.  So the optimal $k$ is when each data point is its own cluster center, but then this is quite useless.  So our goal then becomes to determine the smallest number of clusters $k$ with the lowest error...

The most basic heuristic is to use a graphical method to determine when we are getting diminishing returns for every increase in $k$.  Aptly named the Elbow method, this technique looks for a kink in the error plot.

![](http://i.stack.imgur.com/BzwBY.png)

But as you can probably tell this is a very imprecise method.  How can we improve on this?

### Silhouette

Trying to provide a better metric, the Silhouette was described by Peter Rousseeuw in 1986.  It improves upon the Within Cluster Dispersion by also accounting for how similar each point is to every other point in its cluster (rather than just the centroid) as well as neighboring potential clusters.

We define:

$a(i)$ to be the average dissimilarity of point $i$ to every other data point in its cluster.

$b(i)$ is the lowest dissimilarity of point $i$ to any _other_ cluster (neighboring clusters).

$$ s(i) = 1 - \frac{a(i)}{b(i)} \qquad if \qquad a(i) < b(i) $$
$$ s(i) = 0 \qquad if \qquad a(i) = b(i) $$
$$ s(i) = \frac{b(i)}{a(i)} - 1 \qquad  if \qquad a(i) > b(i)$$

The silhouette is defined to be greatest when $a(i) \ll b(i)$.  And it is lowest when $b(i) \gg a(i)$.

With this definition we can see that the silhouette metric ranges from $-1 \leq s(i) \geq 1$.

In contrast to the elbow method, the silhouette does not monotonically decrease as $k$ increases.  Because of this we can simply vary $k$ and look for when the silhouette is at a maximum.

### Gap Statistic

The Gap Statistic is another technique to estimate the optimal number of clusters in a data set.  It is somewhat unique in that it not only uses the data points, but compares them to a synthetic "null" distribution or random noise.  The Gap statistic can be thought of an automated way to determine the "kink" or "elbow" in the plot of the dispersion.

Also, one of the most apparent weakness of the Silhouette (and most other metrics) is when the true data has no discernible clusters or only one true cluster. The Gap statistic solves some of the issues with the Silhouette by introducing the concept of a reference null dataset to compare performance against.  Let's see how it works!

#### Definition

Just as with the other two methods we will be using the within cluster dispersion ($W_{k}$) to measure "tightness" of clusters. In addition to this measure, we will also utilize a reference null distribution of the data (pure noise) and compare the dispersion on our data vs the reference.  The ideal value of $k$ will then be the value for which $log(W_{k})$ drops furthest below this reference curve:

![](images/gap.png)

And we will define the Gap statistic to be at a maximum exactly when this is so:

$$Gap_{n}(k) = E_{n}^{*}\{log(W_{k})\} - log(W_{k})$$

Where $E_{n}^{*}$ denotes the expectation under a sample size of $n$ from the reference distribution.

#### Motivation

Why use a reference distribution?  Think of this as a sort of *null hypothesis: There are no discernible clusters in the data*

The Gap Statistic is a metric for us to disprove this hypothesis.  But sometimes we won't, and the fact that we have a null hypothesis will allow us to conclude that the ideal number of clusters is 1 (or really 0).

Let us think about how the gap statistic changes as we increase $k$ (assuming $K$ actually clusters) in comparison to the reference distribution (since the reference distribution's dispersion will decrease at a rate proportional to $1/k$:

$log(W_{k})$ should decrease *faster* than the expected rate of the null distribution as we increase $k s.t. k \le K$ since we are in the steep section of the "elbow".

and $log(W_{k})$ should decrease *slower* than the expected rate of the null distribution as we increase $k s.t. k > K$ since we are essentially adding a unnecessary centroid in the middle of an already decent cluster.

#### Reference Distribution

In our formulation, we assume a null model of a single component/cluster and reject it in favor of a $k$-component model ($k > 1$) if there is sufficient evidence contrary.